Patrick BROCKMANN - LSCE (Climate and Environment Sciences Laboratory)<br>
<img align="left" width="40%" src="http://www.lsce.ipsl.fr/Css/img/banniere_LSCE_75.png"><br><br>

<hr>

# Load Modules
#### The following modules are required for this Python Notebook : netCDF4, pandas, numpy, peakutils, randomcolor, ipywidgets

In [1]:
import netCDF4
import pandas as pd
import numpy as np

# Select .cdf file
#### Declare here the .cdf files to be used (160000-13_489mg.cdf in "file = "160000-13_489mg.cdf"")
#### In case you want to explore several result files (but one at once), add the "#" symbol in front of the line not to be used
#### .cdf files must be located in the same folder as the Python Notebook

In [3]:
url = 'https://vesg.ipsl.upmc.fr/thredds/dodsC/IPSLFS/brocksce/Sprectum/'

#file = "160000-13_489mg.cdf"
#file = "160000-10_650mg.cdf"
#file = "Elliott-11_108mg.cdf"
file = "Elliott-21_351mg.cdf"
#file = "Chernozem-23_889mg.cdf"
#file = "Chernozem-9_159mg.cdf"
#file = "Green-1_472mg.cdf"
#file = "Pahokee-2_339mg.cdf"
#file = "Pocahontas-5_256mg.cdf"
#file = "Pocahontas-3_140mg.cdf"
#file = "Suwanee-1_216mg.cdf"
#file = "Suwanee-3_223mg.cdf"
#file = "Vertisol-10_5mg.cdf"
#file = "Vertisol-22_223mg.cdf"

# Dataframe preparation
#### Load the .cdf file as a table
#### Declare the variables (scan_number, point_count and scan_acquisition time)
#### Display the dimension of scan_number (number of scans),  
#### List point_count (m/z) and scan_acquisition_time data (time in seconds)

In [4]:
f = netCDF4.Dataset(url + file)
scan_number = f.dimensions['scan_number'].size
point_count = f.variables['point_count'][:]
scan_acquisition_time = f.variables['scan_acquisition_time'][:]
print(scan_number)
print(point_count)
print(scan_acquisition_time)

3226
[421 410 418 ... 418 416 416]
[ 180.039   180.5431  181.0472 ... 1804.6405 1805.1446 1805.6486]


# Resampling and homogeneisation of the dataframe
#### m/z data in the 50-450 range are resampled

In [5]:
x1 = np.arange(50,450+1,1)

df = pd.DataFrame()
df['m/z'] = x1
df.set_index('m/z', inplace=True)

icurrent = 0
for i in range(0,scan_number):
    #print(i, icurrent, point_count[i])
    x = f.variables['mass_values'][icurrent:icurrent + point_count[i]]
    y = f.variables['intensity_values'][icurrent:icurrent + point_count[i]]
    icurrent = icurrent + point_count[i]
    y1 = np.interp(x1, x, y)
    df[i] = y1

RuntimeError: NetCDF: I/O failure

# Display the dataframe (df) with the intensity of each m/z at every scan number

In [ ]:
df

# Transpose the lines/columns of the df dataframe into the df1 dataframe
#### This allows getting scan numbers as the x axis
#### Rename column "scan_number"  into "scan" and "scan_acquisition_time" column into "Time" - show the df1 table

In [ ]:
df1 = df.T
df1.index.rename('Scan', inplace=True)
df1['Time'] = scan_acquisition_time.data
df1

# Change the index column from scans to time (sec)

In [ ]:
df1 ['Scan'] = df1.index
df1.index = df1.Time
df1

# Display a heatmap of m/z intensity with time (sec)

In [ ]:
import numpy as np

from bokeh.io import output_notebook
from bokeh.plotting import figure, show, output_file
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.models import HoverTool
output_notebook()

mapper = LinearColorMapper(palette="Viridis256", low=0, high=1.5E7)

p = figure(active_scroll='wheel_zoom', tooltips=[("Time", "$x"), ("m/z", "$y"), ("value", "@image")])
p.x_range.range_padding = p.y_range.range_padding = 0

# must give a vector of image data for image parameter
array = [df1.T.rolling(axis=1, center=True, window=100).mean().to_numpy()]

# x, y, dw, dh choosen from data
p.image(image=array, x=0, y=50, dw=30, dh=450, color_mapper=mapper)   

# add titles to axes
p.xaxis.axis_label = "Time (min)"
p.yaxis.axis_label = "m/z"

# Add a colorbar scale
color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="8pt", label_standoff=15, location=(5, 0),)
p.add_layout(color_bar, 'right')

show(p)

# Smoothing the df1  Dataframe values


In [ ]:
df1 = df1.rolling(axis=0, center=True, window=100).mean()
df1.round (1)
df1

# Examination of m/z curves with time (sec)
### Here for m/z 50 to 70 (for x in np.arange(50,70,1):)

In [ ]:
import randomcolor
rand_color = randomcolor.RandomColor()

In [ ]:
p = figure(plot_width=800, plot_height=500, tools='hover,wheel_zoom')
for x in np.arange(50,70,1):
    color = rand_color.generate()
    p.line(x=df1[x].index, y=df1[x], line_width=2, line_color=color[0])
    
# Add axes title
p.xaxis.axis_label = "Time (sec)"
p.yaxis.axis_label = "Intensity (no unit)"

show(p)

# A new column "Temperature" (°C) is created as the index column
#### Temperature values are calculated from the temperature program during pyrolisis (from 5 to 20 min @ 30 °C/min in this case). Only scans acquired during this temperature ramp are conserved

In [ ]:
df1['Temperature'] = df1['Time'].apply(lambda x: 200+(x-300)/2 if (x/60 >= 5) & (x/60 <=20) else -1 )
df1.drop(df1[df1['Temperature'] == -1].index, inplace=True)
df1.index = df1.Temperature.round(1)
df1

# Determination of Tpeak values in m/z intensity curves
#### Import Python modules

In [ ]:
import numpy
import peakutils
from peakutils.plot import plot as pplot
from matplotlib import pyplot
%matplotlib inline

import ipywidgets
from ipywidgets import interact

## Examination of individual m/z intensity curves 
#### A threshold can be defined for improving Tpeak definition. Similarly, you can define a minimal distance between two Tpeaks. By default, the m/z 76 curve is displayed

In [ ]:
# est-il possible d'ajouter infobulles ?

In [ ]:
thres = 0.9
dist = 200

@interact(
    mz=ipywidgets.IntSlider(value=76, min=50, max=450, step=1, continuous_update=False),
    thres=ipywidgets.FloatSlider(value=thres, min=0, max=1, step=0.02, continuous_update=False),
    dist=ipywidgets.FloatSlider(value=dist, min=0, max=1000, step=50, continuous_update=False)
)
def plot(mz, thres, dist):
    x = df1[mz].index
    y = df1[mz].values
    indexes = peakutils.indexes(y, thres=thres, min_dist=dist, thres_abs=False)
    tpeak=[]
    ypos=[]
    if len(indexes) != 0:
        for i in indexes:
            print(mz, i, x[i], y[i])
            tpeak.append(x[i])
            ypos.append(y[i])
            
    pyplot.figure(figsize=(10,6))
    
    pyplot.plot(x, y, color='b', alpha=1)
    pyplot.scatter(tpeak, ypos, marker='o', c='r', alpha=0.5)
    
    pyplot.xlabel("Temperature (°C)")
    pyplot.ylabel("Intensity (no unit)")
    
    pyplot.grid(True)
    axes = pyplot.gca()
    


# Display all m/z intensity curves in the 50-450 range and Tpeak values (as red dots) of each m/z curve. 
#### The threshold for Tpeak definition is set for all m/z

In [ ]:
hover = HoverTool(names=['peaks'])
           
p = figure(plot_width=800, plot_height=500, tools=[hover, 'wheel_zoom'])

for mz in range(50,450+1,1):
    p.line(x=df1[mz].index, y=df1[mz], line_width=1, line_color='blue', alpha=0.1)
    
p.scatter(tpeak, ypos, name='peaks', marker='circle', size=6,
                    color='red', alpha=0.3)
# add titles to axes
p.xaxis.axis_label = "Temperature (°C)"
p.yaxis.axis_label = "Intensity (no unit)"

show(p)

# Prepare a table for .csv export. 
#### The table will contain Tpeak values and maximum intensity defined for each m/z, depending on the threshold defined (a single threshold for all m/z)
#### To note, Tpeak values are not directly comparable to Rock-Eval Tp2 values (see Jacob et al.)

In [ ]:
dfout = pd.DataFrame({'mz': mzList, 'Tpeak': tpeak, 'Intensity' : ypos})
dfout

# Export Tpeak data for m/z intensity curves as a.csv file (out.csv)
#### This file will be generated in the same folder as the Python Notebook and the .cdf files

In [ ]:
dfout.to_csv("out.csv", index=True)